In [1]:
!kaggle datasets download vipoooool/new-plant-diseases-dataset

/bin/bash: line 1: kaggle: command not found


In [31]:
import os
import glob
import keras
import tensorflow as tf
from binary_train_and_eval import gen_dataset

In [36]:
keras.config.enable_unsafe_deserialization()

In [34]:
MODEL_DIR = "out"
model_files = [file for file in os.listdir(MODEL_DIR) if file.endswith("_rgb.keras")]
#model_files = [max(glob.glob(os.path.join("checkpoints", model_dir, "*")), key=os.path.getctime) for model_dir in os.listdir("checkpoints") if model_dir.endswith("_rgb")]
print(model_files)
TEST_DATA_PATH = os.path.join("_data", "combined", "test")
INPUT_SHAPE = (224, 224, 3)

['checkpoints/VGG19_rgb/modelVGG19_rgb.26.keras', 'checkpoints/ConvNeXtLarge_rgb/modelConvNeXtLarge_rgb.36.keras', 'checkpoints/InceptionV3_rgb/modelInceptionV3_rgb.29.keras', 'checkpoints/VisionTransformer_rgb/modelVisionTransformer_rgb.36.keras', 'checkpoints/AlexNet_rgb/modelAlexNet_rgb.30.keras', 'checkpoints/ResNet152V2_rgb/modelResNet152V2_rgb.19.keras', 'checkpoints/MobileNetV3Large_rgb/modelMobileNetV3Large_rgb.36.keras', 'checkpoints/LeNet_rgb/modelLeNet_rgb.05.keras']


In [37]:
models = {file.replace('best', '').replace('_rgb.keras', ''): keras.saving.load_model(file) for file in model_files}

ValueError: bad marshal data (unknown type code)

In [5]:
print(models.keys())

dict_keys(['InceptionV3', 'AlexNet', 'LeNet', 'ResNet152V2', 'MobileNetV3Large', 'VGG19', 'ConvNeXtLarge'])


In [12]:
test_dataset = gen_dataset(TEST_DATA_PATH, batch_size=8, lab=False, input_shape=INPUT_SHAPE)

Found 9199 files belonging to 2 classes.


In [30]:
results = dict()
for name, model in models.items():
    print(f"Evaluating {name}")
    model.compile(optimizer=None, loss='binary_crossentropy', metrics=[
            keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC(),
            tf.keras.metrics.F1Score(),
        ])
    result = model.evaluate(test_dataset)
    results[name] = dict(zip(model.metrics_names, result))

Evaluating InceptionV3
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 127s 106ms/step - auc_8: 0.6693 - binary_accuracy: 0.6365 - loss: 8.9766 - recall_8: 0.6365
Evaluating AlexNet
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 124s 106ms/step - auc_9: 0.7903 - binary_accuracy: 0.7328 - loss: 0.7893 - recall_9: 0.7328
Evaluating LeNet
 743/1150 ━━━━━━━━━━━━━━━━━━━━ 41s 102ms/step - auc_10: 0.6762 - binary_accuracy: 0.6768 - loss: 1.1370 - recall_10: 0.6768

KeyboardInterrupt: 

In [20]:
results

{'InceptionV3': [9.527091026306152,
  0.6293075084686279,
  0.6293075084686279,
  0.6597138047218323],
 'AlexNet': [0.7688920497894287,
  0.741276204586029,
  0.741276204586029,
  0.8005602955818176],
 'LeNet': [0.8527513146400452,
  0.6806174516677856,
  0.6806174516677856,
  0.681216835975647],
 'ResNet152V2': [1.891831398010254,
  0.6833351254463196,
  0.6833351254463196,
  0.7137356400489807],
 'MobileNetV3Large': [5.179934024810791,
  0.6316990852355957,
  0.6316990852355957,
  0.6597292423248291],
 'VGG19': [0.6879483461380005,
  0.6810522675514221,
  0.6810522675514221,
  0.6810522675514221],
 'ConvNeXtLarge': [1.083969235420227,
  0.6703989505767822,
  0.6703989505767822,
  0.7212139368057251]}

In [24]:
results_data = [{'name': name, 'loss': result[0], 'recall': result[1], 'auc': result[3] } for name, result in results.items()]

In [25]:
import pandas as pd
df = pd.DataFrame(results_data)

In [26]:
df

,name,loss,recall,auc
0,InceptionV3,9.527091,0.629308,0.659714
1,AlexNet,0.768892,0.741276,0.800560
2,LeNet,0.852751,0.680617,0.681217
3,ResNet152V2,1.891831,0.683335,0.713736
4,MobileNetV3Large,5.179934,0.631699,0.659729
5,VGG19,0.687948,0.681052,0.681052
6,ConvNeXtLarge,1.083969,0.670399,0.721214


In [29]:
df.set_index('name')
df.loc["Sum"] = df.sum(numeric_only=True)

df.to_latex(float_format="%.4f")

'\\begin{tabular}{llrrr}\n\\toprule\n & name & loss & recall & auc \\\\\n\\midrule\n0 & InceptionV3 & 9.5271 & 0.6293 & 0.6597 \\\\\n1 & AlexNet & 0.7689 & 0.7413 & 0.8006 \\\\\n2 & LeNet & 0.8528 & 0.6806 & 0.6812 \\\\\n3 & ResNet152V2 & 1.8918 & 0.6833 & 0.7137 \\\\\n4 & MobileNetV3Large & 5.1799 & 0.6317 & 0.6597 \\\\\n5 & VGG19 & 0.6879 & 0.6811 & 0.6811 \\\\\n6 & ConvNeXtLarge & 1.0840 & 0.6704 & 0.7212 \\\\\nSum & NaN & 39.9848 & 9.4354 & 9.8344 \\\\\n\\bottomrule\n\\end{tabular}\n'